In [43]:
import pandas as pd

# Load the dataset
data = pd.read_csv("sonar.csv")

In [44]:
# Separate features and the target variable
X = data.iloc[:, :-1] # Features (all columns except the last one)
y = data.iloc[:, -1]  # Target variable (lasst column)

In [45]:
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the data
y = label_encoder.fit_transform(y)

# Print the encoded data
print(y)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [46]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [47]:
X_train = X_train.apply(pd.to_numeric)

In [48]:
import lightgbm as lgb

# Convert the data into LightGBM dataset format
train_data = lgb.Dataset(X_train, label=y_train)

# Set the parameters for the LightGBM classifier
params = {
    'objective': 'binary',
    'metric': 'binary_logloss',
    'boosting_type': 'gbdt',
    'num_leaves': 3,
    'learning_rate': 0.06,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0,
    'force_col_wise': True
}

# Train the LightGBM model
lgb = lgb.train(params, train_data, num_boost_round=100)

print(lgb)

In [49]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Make predictions on the testing set
y_pred = lgb.predict(X_test)
y_pred = [1 if pred >= 0.5 else 0 for pred in y_pred]  # Convert probabilities to class labels

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, pos_label=1)
recall = recall_score(y_test, y_pred, pos_label=1)
f1 = f1_score(y_test, y_pred, pos_label=1)

# Print the evaluation metrics
print("Accuracy:", accuracy)
print("Precision (Mines):", precision)
print("Recall (Mines):", recall)
print("F1-Score (Mines):", f1)

Accuracy: 0.8571428571428571
Precision (Mines): 0.8
Recall (Mines): 0.8
F1-Score (Mines): 0.8000000000000002


## GridSearchCV

In [50]:
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMClassifier

# defining parameter range
param_grid = {'n_estimators': [100, 200, 300],
              'learning_rate': [0.01, 0.1, 1.0],
              'max_depth': [3, 4, 5]}

grid = GridSearchCV(LGBMClassifier(), param_grid, refit=True, verbose=6)


# fitting the model for grid search
grid.fit(X_train, y_train)

Fitting 5 folds for each of 27 candidates, totalling 135 fits
[CV 1/5] END learning_rate=0.01, max_depth=3, n_estimators=100;, score=0.818 total time=   0.0s
[CV 2/5] END learning_rate=0.01, max_depth=3, n_estimators=100;, score=0.788 total time=   0.0s
[CV 3/5] END learning_rate=0.01, max_depth=3, n_estimators=100;, score=0.818 total time=   0.0s
[CV 4/5] END learning_rate=0.01, max_depth=3, n_estimators=100;, score=0.697 total time=   0.0s
[CV 5/5] END learning_rate=0.01, max_depth=3, n_estimators=100;, score=0.697 total time=   0.0s
[CV 1/5] END learning_rate=0.01, max_depth=3, n_estimators=200;, score=0.848 total time=   0.1s
[CV 2/5] END learning_rate=0.01, max_depth=3, n_estimators=200;, score=0.788 total time=   0.0s
[CV 3/5] END learning_rate=0.01, max_depth=3, n_estimators=200;, score=0.848 total time=   0.0s
[CV 4/5] END learning_rate=0.01, max_depth=3, n_estimators=200;, score=0.727 total time=   0.1s
[CV 5/5] END learning_rate=0.01, max_depth=3, n_estimators=200;, score=0.7

GridSearchCV(estimator=LGBMClassifier(),
             param_grid={'learning_rate': [0.01, 0.1, 1.0],
                         'max_depth': [3, 4, 5],
                         'n_estimators': [100, 200, 300]},
             verbose=6)

In [51]:
# Best hyperparameters
print("Best Hyperparameters: ", grid.best_params_)

# Best model (estimator)
best_model = grid.best_estimator_

Best Hyperparameters:  {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 200}


In [52]:
grid_predictions = grid.predict(X_test)
from sklearn.metrics import classification_report
# print classification report
print(classification_report(y_test, grid_predictions))
print(accuracy_score(y_test, grid_predictions))

              precision    recall  f1-score   support

           0       0.92      0.81      0.86        27
           1       0.72      0.87      0.79        15

    accuracy                           0.83        42
   macro avg       0.82      0.84      0.83        42
weighted avg       0.85      0.83      0.84        42

0.8333333333333334


## Bayesian

In [53]:
!pip install scikit-optimize

In [54]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [55]:
from sklearn.metrics import accuracy_score
from skopt import BayesSearchCV

# Define the objective function to optimize
def objective_function(learning_rate, n_estimators, max_depth):
    # Create the gradient boosting classifier with the hyperparameters
    model = LGBMClassifier(learning_rate=learning_rate, n_estimators=int(n_estimators), max_depth=int(max_depth))

    # Fit the model to the training data
    model.fit(X_train, y_train)

    # Predict on the test data
    y_pred = model.predict(X_test)

    # Calculate and return the accuracy score
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

# Define the search space for hyperparameters
search_spaces = {'learning_rate': (0.01, 0.1),
                 'n_estimators': (100, 300),
                 'max_depth': (3, 5)}

# Perform Bayesian optimization
optimizer = BayesSearchCV(estimator=LGBMClassifier(), search_spaces=search_spaces, n_iter=50, cv=5, random_state=42)
optimizer.fit(X_train, y_train)

# Print the best hyperparameters and corresponding score
print("Best Hyperparameters: ", optimizer.best_params_)
print("Best Score: ", optimizer.best_score_)


Best Hyperparameters:  OrderedDict([('learning_rate', 0.07983273405390498), ('max_depth', 3), ('n_estimators', 300)])
Best Score:  0.9030303030303031


In [56]:
y_pred = optimizer.predict(X_test)
bayes_predictions = optimizer.predict(X_test)
from sklearn.metrics import classification_report
# print classification report
print(classification_report(y_test, bayes_predictions))
print(accuracy_score(y_test, bayes_predictions))

              precision    recall  f1-score   support

           0       0.88      0.81      0.85        27
           1       0.71      0.80      0.75        15

    accuracy                           0.81        42
   macro avg       0.79      0.81      0.80        42
weighted avg       0.82      0.81      0.81        42

0.8095238095238095


## Tree-structured Parzen Estimators

In [57]:
from lightgbm import LGBMClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from hyperopt import hp, tpe, fmin, Trials

# Generate a synthetic dataset for classification
X, y = make_classification(n_samples=1000, random_state=42)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the objective function to minimize
def objective(params):
    learning_rate = params['learning_rate']
    n_estimators = params['n_estimators']
    max_depth = params['max_depth']

    model = LGBMClassifier(learning_rate=learning_rate, n_estimators=n_estimators, max_depth=max_depth)
    model.fit(X_train, y_train)

    accuracy = model.score(X_test, y_test)
    return {
        'loss': -accuracy,
        'status': 'ok',
        'accuracy': accuracy
    }

# Define the search space for hyperparameters
space = {
    'learning_rate': hp.loguniform('learning_rate', -5, 0),
    'n_estimators': hp.choice('n_estimators', range(100, 301, 50)),
    'max_depth': hp.choice('max_depth', range(3, 6))
}

# Perform hyperparameter tuning with TPE
trials = Trials()
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=50, trials=trials)

# Print the best hyperparameters and accuracy
print("Best Hyperparameters: ", best)
best_trial = trials.best_trial
print("Best Accuracy: ", -best_trial['result']['loss'])


  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

2023-07-10 18:04:58,829 - hyperopt.tpe - INFO - build_posterior_wrapper took 0.001895 seconds
2023-07-10 18:04:58,832 - hyperopt.tpe - INFO - TPE using 0 trials


  2%|▏         | 1/50 [00:00<00:05,  8.50trial/s, best loss: -0.9]

2023-07-10 18:04:58,951 - hyperopt.tpe - INFO - build_posterior_wrapper took 0.003254 seconds
2023-07-10 18:04:58,953 - hyperopt.tpe - INFO - TPE using 1/1 trials with best loss -0.900000
2023-07-10 18:04:59,008 - hyperopt.tpe - INFO - build_posterior_wrapper took 0.002008 seconds
2023-07-10 18:04:59,010 - hyperopt.tpe - INFO - TPE using 2/2 trials with best loss -0.900000


  6%|▌         | 3/50 [00:00<00:06,  7.82trial/s, best loss: -0.9]

...

 96%|█████████▌| 48/50 [00:34<00:00,  6.84trial/s, best loss: -0.915]

2023-07-10 18:05:33,018 - hyperopt.tpe - INFO - build_posterior_wrapper took 0.001896 seconds
2023-07-10 18:05:33,020 - hyperopt.tpe - INFO - TPE using 48/48 trials with best loss -0.915000


 98%|█████████▊| 49/50 [00:34<00:00,  6.92trial/s, best loss: -0.915]

2023-07-10 18:05:33,158 - hyperopt.tpe - INFO - build_posterior_wrapper took 0.001891 seconds
2023-07-10 18:05:33,161 - hyperopt.tpe - INFO - TPE using 49/49 trials with best loss -0.915000


100%|██████████| 50/50 [00:34<00:00,  1.45trial/s, best loss: -0.915]
Best Hyperparameters:  {'learning_rate': 0.18249376796705738, 'max_depth': 1, 'n_estimators': 3}
Best Accuracy:  0.915


## Simulated Annealing

In [58]:
!pip install simanneal

In [59]:
from lightgbm import LGBMClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from scipy.optimize import basinhopping

# Generate a synthetic dataset for classification
X, y = make_classification(n_samples=1000, random_state=42)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the objective function to minimize
def objective(params):
    learning_rate = params[0]
    n_estimators = int(params[1])
    max_depth = int(params[2])

    model =LGBMClassifier(learning_rate=learning_rate, n_estimators=n_estimators, max_depth=max_depth)
    model.fit(X_train, y_train)

    accuracy = model.score(X_test, y_test)
    return -accuracy

# Define the bounds for hyperparameters
bounds = [(0.001, 0.1), (100, 300), (3, 5)]

# Perform hyperparameter tuning with Simulated Annealing
result = basinhopping(func=objective, x0=[0.01, 150, 4], minimizer_kwargs={"bounds": bounds})

# Print the best hyperparameters and accuracy
best_params = result.x
best_accuracy = -result.fun
print("Best Hyperparameters: ", best_params)
best_trial = trials.best_trial
print("Best Accuracy: ", -best_trial['result']['loss'])


Best Hyperparameters:  [1.0e-02 1.5e+02 4.0e+00]
Best Accuracy:  0.915


# Randomized Search

In [60]:
from sklearn.model_selection import RandomizedSearchCV
from lightgbm import LGBMClassifier
from scipy.stats import uniform

# Define the hyperparameter search space
param_dist = {
    'learning_rate': uniform(0.01, 1),
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5]
}

# Create an LGBMClassifier object
lgb = LGBMClassifier()

# Create a RandomizedSearchCV object
random_search = RandomizedSearchCV(
    estimator=lgb,
    param_distributions=param_dist,
    n_iter=10,  # Number of random combinations to try
    scoring='accuracy',  # Evaluation metric
    cv=5,  # Number of cross-validation folds
    random_state=42
)

# Perform random search
random_search.fit(X_train, y_train)

# Print the best hyperparameters and the corresponding evaluation score
print("Best Hyperparameters:", random_search.best_params_)
print("Best Score:", random_search.best_score_)


Best Hyperparameters: {'learning_rate': 0.03058449429580245, 'max_depth': 4, 'n_estimators': 200}
Best Score: 0.91375


In [61]:
random_predictions = random_search.predict(X_test)
from sklearn.metrics import classification_report
# print classification report
print(classification_report(y_test, random_predictions))
print(accuracy_score(y_test, random_predictions))

              precision    recall  f1-score   support

           0       0.84      0.97      0.90        93
           1       0.97      0.84      0.90       107

    accuracy                           0.90       200
   macro avg       0.90      0.90      0.90       200
weighted avg       0.91      0.90      0.90       200

0.9


# Genetic Algorithms

In [62]:
import random
from sklearn.model_selection import cross_val_score
from lightgbm import LGBMClassifier

# Define the hyperparameter search space
param_space = {
    'learning_rate': [0.01, 0.1, 1.0],
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5]
}

# Set the population size, number of generations, and mutation rate
population_size = 10
num_generations = 5
mutation_rate = 0.1

# Define the fitness function to evaluate a hyperparameter configuration
def fitness_function(params):
    lgb = LGBMClassifier(learning_rate=params['learning_rate'], n_estimators=params['n_estimators'], max_depth=params['max_depth'])
    scores = cross_val_score(lgb, X_train, y_train, cv=5, scoring='accuracy')
    return scores.mean()


# Initialize the population with random hyperparameter configurations
population = []
for _ in range(population_size):
    params = {
        'learning_rate': random.choice(param_space['learning_rate']),
        'n_estimators': random.choice(param_space['n_estimators']),
        'max_depth': random.choice(param_space['max_depth'])
    }
    population.append(params)

# Iterate through generations
for _ in range(num_generations):
    # Evaluate the fitness of each individual in the population
    fitness_scores = [fitness_function(individual) for individual in population]

    # Select parents for reproduction (tournament selection)
    parents = []
    for _ in range(population_size // 2):
        tournament = random.choices(population, k=2, weights=fitness_scores)
        parent = max(tournament, key=fitness_function)
        parents.extend(tournament)

    # Perform crossover to create offspring
    offspring = []
    for i in range(0, population_size, 2):
        parent1 = parents[i]
        parent2 = parents[i + 1]
        child = {
            'learning_rate': random.choice([parent1['learning_rate'], parent2['learning_rate']]),
            'n_estimators': random.choice([parent1['n_estimators'], parent2['n_estimators']]),
            'max_depth': random.choice([parent1['max_depth'], parent2['max_depth']])
        }
        offspring.append(child)

    # Perform mutation on the offspring
    for individual in offspring:
        if random.random() < mutation_rate:
            individual['learning_rate'] = random.choice(param_space['learning_rate'])
        if random.random() < mutation_rate:
            individual['n_estimators'] = random.choice(param_space['n_estimators'])
        if random.random() < mutation_rate:
            individual['max_depth'] = random.choice(param_space['max_depth'])

    # Replace the old population with the new offspring
    population = offspring

# Evaluate the final population
fitness_scores = [fitness_function(individual) for individual in population]

# Find the best hyperparameter configuration
best_index = max(range(len(fitness_scores)), key=lambda i: fitness_scores[i])
best_params = population[best_index]

print("Best Hyperparameters:", best_params)
print("Best Accuracy:", fitness_scores[best_index])


Best Hyperparameters: {'learning_rate': 0.1, 'n_estimators': 200, 'max_depth': 5}
Best Accuracy: 0.905
